In [1]:
from data_processing import *
from feature_extraction import *
from transformations import *

In [2]:
base = "D:/Users/Anthony/Documents/Kaggle-Competitions/KaggleEEG/data_processing/"
train1_path = join(base, 'test')

In [13]:
class Processor(object):

    def __init__(self, list_of_functions, dtrend=None):
        self.list_of_functions = list_of_functions
        self.dtrend = dtrend

    def process_folder(self, train_path):
        """ Apply function to all files in
        """
        seizure_df = pd.DataFrame()
        failures = []
        results = []
        print(train_path)
        for patient_path in train_path:
            # This is how I speed up processing 4x by making full use of all cores in the CPUs.
            values = [delayed(self.process_file)('\\'.join([patient_path, f])) for f in listdir(patient_path) if isfile('/'.join([patient_path, f]))]
            result = compute(*values, get=dask.multiprocessing.get)
            results.append(result)
        return results

    def process_file(self, fname):
        """ Apply list of functions to file.

        Each function should return a dataframe with x columns and 1 row. The number of columns is equal to features
        extracted.

        :param fname:
        :param list_of_functions:
        :param dtrend (string):  Must be  'None', 'mean', 'median'
        :return:
        """
        df, sampling_rate, sequence = load_data(join(fname))
        df = self.normalize(df)
        # Determine if this is an inter or preictal dataset and put in corresponding bucket.
        split_string = fname.split('/').pop().replace('.', '_').split('_')
        feature_df_list = []
        for func in self.list_of_functions:
            # Process function and append index columns
            func_result_df = func(df)
            feature_df_list.append(func_result_df)
        feature_df = pd.concat(feature_df_list, 1)
        feature_df = self.append_index(feature_df, split_string)
        return feature_df

    def append_index(self, df, split_string):
        """ Append data set identifier and set index to identifier"""
        path, patient_id = split_string[0].split('\\')
        df['patient'] = patient_id
        df['dataset_id'] = split_string[1]
        df['pre_ictal'] = split_string[2]
        
        return df

    def normalize(self, df):
        """ Normalize data frame.
        """
        return df

In [14]:
test = Processor([detect_dropout])

In [15]:
res = test.process_folder([train1_path])

['D:/Users/Anthony/Documents/Kaggle-Competitions/KaggleEEG/data_processing/test']


In [16]:
res[0]

(         df_sum patient dataset_id pre_ictal
 0  84182.429688       1          1         0,
      df_sum patient dataset_id pre_ictal
 0  107245.0       1          1         1,
          df_sum patient dataset_id pre_ictal
 0  54031.632812       1          2         0,
           df_sum patient dataset_id pre_ictal
 0  100579.195312       1          2         1,
          df_sum patient dataset_id pre_ictal
 0  86772.695312       1          3         0)

In [17]:
pd.concat(res[0])

,df_sum,patient,dataset_id,pre_ictal
0,84182.429688,1,1,0
0,107245.000000,1,1,1
0,54031.632812,1,2,0
0,100579.195312,1,2,1
0,86772.695312,1,3,0
